#### Importing a csv file into a postgresql database

#### STEPS

- Import the CSV file into a pandas df
- Clean the table name and remove all extra symbols, spaces, capital letters
- clean the column headers and remove all extra symbols, space, capital letters
- write the create table SQL statement
- import the data into the db

In [1]:
#import libraries

import os
import numpy as np
import pandas as pd
import psycopg2

In [ ]:
!ls

In [8]:
df = pd.read_csv("Downloads/Customer Contracts$.csv")
df.head()

,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-2019,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-2015,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-2012,12-20-2015,6.66,No,Yes


In [11]:
#Clean Table Names
# lower case letters
# remove all white spaces
# repalce -,/,\\,$, with __

file = "Customer Contracts$"

clean_tbl_name = file.lower().replace(" ","_").replace("?","") \
                    .replace("-","_").replace(r"/","_").replace("\\","_").replace("%","") \
                    .replace(")","").replace(r"(","").replace("$","")

clean_tbl_name


'customer_contracts'

In [15]:
#Clean Header Names
# lower case letters
# remove all white spaces
# repalce -,/,\\,$, with __


df.columns = [x.lower().replace(" ","_").replace("?","") \
                    .replace("-","_").replace(r"/","_").replace("\\","_").replace("%","") \
                    .replace(")","").replace(r"(","").replace("$","") for x in df.columns]
df.columns


Index(['customer_name', 'start_date', 'end_date', 'contract_amount_m',
       'invoice_sent', 'paid'],
      dtype='object')

In [19]:
create table customer_contracts
(
    customer_name      varchar,
    start_date         varchar,
    end_date           varchar,
    contract_amount_m  float,
    invoice_sent       varchar,
    paid               varchar,
);

SyntaxError: invalid syntax (4128993740.py, line 1)

In [17]:
df.dtypes

customer_name         object
start_date            object
end_date              object
contract_amount_m    float64
invoice_sent          object
paid                  object
dtype: object

In [18]:
replacements = {
    'object':'varchar',
    'float64':'float',
    'int64':'int',
    'datetime64':'timestamp',
    'timedelta64[ns]': 'varchar'
}
replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'int',
 'datetime64': 'timestamp',
 'timedelta64[ns]': 'varchar'}

In [21]:
col_str = ", ".join("{} {}".format(n, d) for (n,d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'customer_name varchar, start_date varchar, end_date varchar, contract_amount_m float, invoice_sent varchar, paid varchar'

In [45]:
# Open a database connection 
# update user,password,host,port and database name

conn = psycopg2.connect(user="postgres",
            password = "password", 
            host = "127.0.0.1",
            port = "5432",
            database = "dbase")
cursor = conn.cursor()
print('opened database successfully')


opened database successfully


In [46]:
#drop tables with same name

cursor.execute("drop table if exists customer_contracts;")

In [47]:
#create table
cursor.execute("create table customer_contracts (customer_name varchar, start_date varchar, end_date varchar, contract_amount_m float,invoice_sent varchar, paid varchar)")

In [49]:
#insert values to table 

#save df to csv
df.to_csv('customer_contracts.csv',header=df.columns,index=False,encoding='utf8')

#open the csv file, save it as an object, and upload to db
my_file = open('customer_contracts.csv')
print('file opened in memory')

file opened in memory


In [50]:
#upload to db

SQL_STATEMENT = """
COPY customer_contracts FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
"""

cursor.copy_expert(sql=SQL_STATEMENT,file=my_file)
print('file copied to db')

file copied to db


In [51]:
cursor.execute("grant select on table customer_contracts to public")
conn.commit()

cursor.close()
print('table customer_contracts imported to db completed')

table customer_contracts imported to db completed


In [54]:
# Only if you want to give access to all the user who has access to your database
# cursor.execute("grant select on table customer_contracts to public")
# conn.commit()

# cursor.close()
# print('table customer_contracts imported to db completed')